# Import Module

In [1]:
# import module

import pandas as pd
import numpy as np
import scipy.io as sio

import os
import glob
import time
import math
from IPython.display import clear_output

import torch
import torch.nn.functional as F
import numpy as np
from torch import optim
import matplotlib.pyplot as plt
import cv2

from torch.autograd import Variable
from torchsampler import ImbalancedDatasetSampler
from itertools import compress

import gc

# for loss function
from typing import Optional
from kornia.utils.one_hot import one_hot

print(torch.cuda.is_available())
print(torch.__version__)

from models import DP_UNet, DP_resUNet

from patch_function import makepatches_overlay, \
makepatches_overlay_normap, \
patchesback_overlay, \
find_ksp_andpadding, \
pad_back, \
pad_for_256, \
pad_back_256,\
patch_inference


from data_preprocess_and_loader import Dataset, Dataset_val, Dataset_out

from losses_unet3d import DiceLoss, GeneralizedDiceLoss, compute_per_channel_dice

from train_and_val import train, validation,\
inference_and_output

True
1.10.2


# Define Datasets

In [3]:
# Define Datasets
path_tra_1 = 'E:/BrainTumor_TCVGH/Dataset/Dataset_tcvgh/1_tra'
list_tra_1 = os.listdir(path_tra_1)
for i in range(len(list_tra_1)):
    list_tra_1[i] = path_tra_1+'/'+list_tra_1[i]
path_tra_2 = 'F:/Datasets_TPVGH/Dataset_tpvgh/1_tra'
list_tra_2 = os.listdir(path_tra_2)
for i in range(len(list_tra_2)):
    list_tra_2[i] = path_tra_2+'/'+list_tra_2[i]    
list_tra = np.append(list_tra_1,list_tra_2)


path_val_1 = 'E:/BrainTumor_TCVGH/Dataset/Dataset_tcvgh/2_val'
list_val_1 = os.listdir(path_val_1)
for i in range(len(list_val_1)):
    list_val_1[i] = path_val_1+'/'+list_val_1[i]
path_val_2 = 'F:/Datasets_TPVGH/Dataset_tpvgh/2_val'
list_val_2 = os.listdir(path_val_2)
for i in range(len(list_val_2)):
    list_val_2[i] = path_val_2+'/'+list_val_2[i]    
list_val = np.append(list_val_1,list_val_2)

    
path_ts_1 = 'E:/BrainTumor_TCVGH/Dataset/Dataset_tcvgh/3_trial'
list_ts_1 = os.listdir(path_ts_1)
for i in range(len(list_ts_1)):
    list_ts_1[i] = path_ts_1+'/'+list_ts_1[i]
path_ts_2 = 'F:/Datasets_TPVGH/Dataset_tpvgh/3_trial'
list_ts_2 = os.listdir(path_ts_2)
for i in range(len(list_ts_2)):
    list_ts_2[i] = path_ts_2+'/'+list_ts_2[i]    
list_ts = np.append(list_ts_1,list_ts_2)

In [4]:
# 設置dataloader output
train_data = Dataset_out(list_tra)
tra_loader = torch.utils.data.DataLoader(
    train_data,           
    batch_size = 1,                 
    shuffle = False)
val_data = Dataset_out(list_val)
val_loader = torch.utils.data.DataLoader(
    val_data,           
    batch_size = 1,                 
    shuffle = False)
ts_data = Dataset_out(list_ts)
ts_loader = torch.utils.data.DataLoader(
    ts_data,           
    batch_size = 1,                 
    shuffle = False)

Dataloader = [val_loader, ts_loader]

# Initialize the Model

In [5]:
# Initialization

model = DP_resUNet(img_channels = 2, n_classes = 2).cuda()
#model = DP_UNet(img_channels = 2, n_classes = 2).cuda()

# define optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 10, gamma=0.95)

# loss function
criterion_DICE = DiceLoss()
criterion_GDL = GeneralizedDiceLoss()

loss_fun = (criterion_DICE,criterion_GDL,compute_per_channel_dice)

# Load Trained Model

In [6]:
# load model
PATH = 'F:/Experiments/3D/3DresDP_tc+tp_fin_s24/model_tmp/best_val.tar'
own_state = torch.load(PATH)
model.load_state_dict(own_state)
model.eval();

# Start to output

In [8]:
sav_path = 'F:/Experiments/3D/3DresDP_tc+tp_fin_s24/results_bestval_64645'
str_ = ['2_val','3_trial']

for i in range(2):

    path = sav_path + '/' + str_[i]    
    f = open(sav_path + '/loss_curve.txt', 'a')
    inference_and_output(save_mode=True,
                        data_loader = Dataloader[i],
                        model = model,
                        loss_fun = loss_fun,
                        f = f,
                        kernel_size = (256,256,24),
                        stride = (64,64,5),
                        inf_pnum = 10,
                        sav_path = path)

|    10/  284 batches | ms/batch 504150.65 | loss_dice  0.10 | loss_gdl  0.19 | 
|    20/  284 batches | ms/batch 340905.53 | loss_dice  0.13 | loss_gdl  0.25 | 
|    30/  284 batches | ms/batch 431443.65 | loss_dice  0.13 | loss_gdl  0.25 | 
|    40/  284 batches | ms/batch 1242983.09 | loss_dice  0.15 | loss_gdl  0.30 | 
|    50/  284 batches | ms/batch 820450.43 | loss_dice  0.15 | loss_gdl  0.28 | 
|    60/  284 batches | ms/batch 1760086.21 | loss_dice  0.16 | loss_gdl  0.31 | 
|    70/  284 batches | ms/batch 1311780.41 | loss_dice  0.17 | loss_gdl  0.33 | 
|    80/  284 batches | ms/batch 1672057.50 | loss_dice  0.17 | loss_gdl  0.33 | 
|    90/  284 batches | ms/batch 1617142.24 | loss_dice  0.17 | loss_gdl  0.33 | 
|   100/  284 batches | ms/batch 1801496.47 | loss_dice  0.18 | loss_gdl  0.34 | 
|   110/  284 batches | ms/batch 583654.80 | loss_dice  0.18 | loss_gdl  0.34 | 
|   120/  284 batches | ms/batch 92096.88 | loss_dice  0.17 | loss_gdl  0.32 | 
|   130/  284 batches |

# TMP

In [9]:
sav_path = 'F:/Experiments/3D/3DresDP_tc+tp_fin_s24/results_bestval_646412'
str_ = ['2_val','3_trial']

for i in range(2):

    path = sav_path + '/' + str_[i]    
    f = open(sav_path + '/loss_curve.txt', 'a')
    inference_and_output(save_mode=True,
                        data_loader = Dataloader[i],
                        model = model,
                        loss_fun = loss_fun,
                        f = f,
                        kernel_size = (256,256,24),
                        stride = (64,64,12),
                        inf_pnum = 10,
                        sav_path = path)

|    10/  284 batches | ms/batch 278816.11 | loss_dice  0.11 | loss_gdl  0.21 | 
|    20/  284 batches | ms/batch 202341.91 | loss_dice  0.14 | loss_gdl  0.27 | 
|    30/  284 batches | ms/batch 254592.80 | loss_dice  0.13 | loss_gdl  0.26 | 
|    40/  284 batches | ms/batch 541778.95 | loss_dice  0.16 | loss_gdl  0.30 | 
|    50/  284 batches | ms/batch 396862.11 | loss_dice  0.15 | loss_gdl  0.29 | 
|    60/  284 batches | ms/batch 768272.19 | loss_dice  0.16 | loss_gdl  0.31 | 
|    70/  284 batches | ms/batch 588260.55 | loss_dice  0.17 | loss_gdl  0.33 | 
|    80/  284 batches | ms/batch 730492.87 | loss_dice  0.18 | loss_gdl  0.34 | 
|    90/  284 batches | ms/batch 689943.77 | loss_dice  0.17 | loss_gdl  0.33 | 
|   100/  284 batches | ms/batch 766464.69 | loss_dice  0.18 | loss_gdl  0.34 | 
|   110/  284 batches | ms/batch 283901.87 | loss_dice  0.18 | loss_gdl  0.34 | 
|   120/  284 batches | ms/batch 80393.02 | loss_dice  0.17 | loss_gdl  0.32 | 
|   130/  284 batches | ms/ba